# MCU

今年 7 月，韩松团队发表的论文 “MCUNet: Tiny Deep Learning on IoT Devices” 中提出了 MCUNet 方案，可以通过高效的网络架构搜索（TinyNAS）和轻量推理引擎（TinyEngine）的联合设计，实现在 MCU（Microcontroller Unit 微控制单元，又称单片机）上运行 ImageNet 级别的推理。该论文第一作者林己在接受 DeepTech 采访时表示，“我们发现，用我们的 MCUNet 方案，基本上相对轻量级的应用，在手机上跑得比较快的应用，都可以部署到单片机上。”

该论文显示，相比谷歌的 TF-Lite Micro 与 ARM 的 CMSIS-NN 的传统方案，使用 MCUNet 进行学习推理所需要的内存可以减少为原来的 29%，同时推理速度能够提升 1.7-3.3 倍。

http://tinyml.mit.edu/

https://hanlab.mit.edu/projects/tinyml/tinyTL/

MCUNet 实现 IoT 设备上的微型机器学习
一直以来，深度学习在 IoT 微型设备上的运行难点在于，单片机的硬件资源限制。因为单片机的内存资源仅为手机、GPU 的几千分之一，这导致原本在手机和 GPU 上能够完美运行的 AI 方案，根本无法放入单片机中（模型大小过大），运行就更是别想（activation过大）。先前的模型压缩、网络设计相关工作，主要是优化减小神经网络的计算量或者模型大小，没有考虑到运行时神经网络的 activation 大小，这导致优化过的神经网络也难以满足单片机的资源限制。

对此，韩松团队开发的 MCUNet 方案，采用 TinyNAS 和 TinyEngine 结合的方式，能够合理规划单片机上仅有的内存，并进行高效的推理运算。

其中，TinyNAS 需要通过两个步骤完成对单片机有限的空间进行合理规划，以达到提升最终模型精确度的目的。

第一步，自动搜索神经网络空间来适配不同硬件的资源限制，找到可以满足硬件资源限制的最高精度的搜索空间；第二步，根据不同搜索空间中神经网络计算量的分布，选择特定单片机的自由搜索空间进行网络架构的搜索。

搜索空间时，需要输入分辨率和网络宽度进行搜索。大概有 108 个可能的搜索空间配置，每个搜索空间配置包含 3.3×1025 个子网络。

韩松团队认为，在内存约束下更有可能生成高 FLOPS 模型的设计空间可以提供更高的模型容量，因此有可能实现高精度。对比红色和黑色，使用红色空间进行神经体系结构搜索可达到 78.7% 的最终精度，而黑色空间则为 74.2%(在 ImageNet 100 上)。

MIT提出微型深度学习技术MCUNet可在物联网设备高效运行

图 | TinyNAS 通过分析不同搜索空间的 FLOP CDF 来选择最佳搜索空间

经过 TinyNAS 的优化，确保单片机在硬件资源最佳分配的状态下使用 TinyEngine 展开推理运算。

在推理运算方面，传统的机器学习方案 TF-Lite Micro 与 CMSIS-NN 采用的是基于解释器的运行框架。在推理运行的同时进行网络结构的解释，所以需要将神经网络操作中所有可能用到的算子以及支持所有上级的代码全部储存，这必须要有足够大的内存才能够承载。另外，解释型框架是在运行期间对神经网络的结构进行解释，这一动作也将增加运行的负担。

这种 “甜蜜的负担” 对于 “身强力壮的巨人” 手机和 GPU 等内存足够大的设备并无大碍，但对于单片机这种自身条件有限的 “小家伙” 来说，简直寸步难行，甚至有将其压垮的架势。

由此，韩松团队为 “小家伙” 量身定制了新的方案 ——MCUNet，林己向 DeepTech 介绍称，MCUNet 使用 TinyEngine 框架，将神经网络的编译和执行步骤分开进行。编译阶段明确决定好内存的调度、执行过程中的必备动作以及所需算子，在执行过程中，微型处理器上只需要对当前神经网络所执行的任务需要的算子进行储存，内存调度上也无需进行动态处理。做到节省解释时间的同时，也让内存资源本就紧张的单片机能够 “松一口气”，让更大模型神经网络的运行看到一丝希望。

实验结果显示，使用 TinyEngine 运行之下的推理，与传统框架 TF-Lite Micro 对比，推理速度提升了 3 倍，内存占用缩减为1/4，在 ImageNet 上的准确率则提高了 17%。

MIT提出微型深度学习技术MCUNet可在物联网设备高效运行

图 | TinyEngine 与传统框架推理效率、内存占用峰值、精确度对比

值得注意的是，在该论文发表之后，韩松团队又加入了 in-place depthwise convolution 技术，结合 TinyNAS 可以进一步缩小神经网络运行时的最大内存，同时降低神经网络运行的硬件需求，实现 MCUNet 方案推理运算下效率和准确率的进一步提升。另外，林己还提到，“现在我们还加入了目标检测相关的应用，比如可以用来检测人、车以及人有没有戴口罩等，而此前主要关注分类方面的应用”。

MCUNet 诞生于新冠疫情期间
“整体研发尤其后期，是在疫情下进行的，同学们都是在隔离的条件下独立完成研发，团队通过远程进行交流，其实做得很辛苦。”

谈及 MCUNet 的研发过程，韩松表示他印象最深刻的是疫情期间团队成员在无法面对面交流的情况下进行研发，挑战很大。

前文提到，提出 MCUNet 方案的论文已入选今年的 NeurIPS spotlight。其实，除此之外，韩松团队今年入选 NeurIPS 的还有另外两篇，分别为《TinyTL：降低内存占用而非参数量用于高效的设备上学习》和《可微分数据增强：训练数据高效的生成对抗网络》。在此不对另外两篇论文进行展开，简单了解下该团队中 MCUNet 研发的主要成员。[1]

[1]: http://www.mittrchina.com/news/5866